In [1]:
from utilities import utils, train_utils
from models.LSTM_AE import LSTM_AE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

import numpy as np

import os
import glob
import torch
import tqdm
import zipfile

In [2]:
experiment_config = './experiment_configs/LSTM_AE_exp00.yml'

split_seed = 42

data_url = 'https://mega.nz/file/sMpHGCbA#b0eT2MQjOJzvicnGzESXDF2Z5KXznIfw4OdLQRDA21s'
data_save_path = 'data'
configs = utils.read_yaml(experiment_config)
device = train_utils.get_device()

In [ ]:
utils.download_mega(data_url, data_save_path)
with zipfile.ZipFile('./data/environmental_embeddings_1k-msg-pack-2023-10-17.zip', 'r') as zip_ref:
    zip_ref.extractall(data_save_path)

In [3]:
# TODO: wrap this in utils
paths = sorted(glob.glob('./data/environmental_embeddings_0001/0001/*.msgpack'))

X = []
for path in tqdm.tqdm(paths):
    pos_emb, neg_emb = utils.read_embedding_data(path)
    X.append(pos_emb)
X = np.concatenate(X, axis=0)

Xtr, Xvl = train_test_split(X, test_size=0.2, random_state=split_seed)

100%|██████████| 1000/1000 [00:04<00:00, 201.66it/s]


In [4]:
mean = np.mean(Xtr)
var = np.var(Xtr)

Xtr_scl = (Xtr - mean) / var
Xvl_scl = (Xvl - mean) / var

In [5]:
train_data = torch.tensor(Xtr_scl, dtype=torch.float32)
val_data = torch.tensor(Xvl_scl, dtype=torch.float32)

train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=configs['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=configs['batch_size'], shuffle=False)

In [6]:
# TODO: clean this
if configs['out_activ'] == 'tanh':
    out_activ = torch.nn.Tanh()
elif configs['out_activ'] == 'sigmoid':
    out_activ = torch.nn.Sigmoid()

model = LSTM_AE(
    input_dim=configs['input_dim'],
    encoding_dim=configs['encoding_dim'],
    h_dims=configs['h_dims'],
    out_activ=out_activ
)

model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=configs['lr'])
criterion = torch.nn.MSELoss(reduction='sum')

In [7]:
train_utils.train_loop(
    train_loader=train_loader,
    val_loader=val_loader,
    model=model,
    criterion=criterion,
    device=device,
    optimizer=optimizer,
    epochs=configs['epochs'],
    save_path=configs['save_path']
)

val loss: 45419.074: 100%|██████████| 7/7 [00:01<00:00,  6.04it/s]


Weight saved: epoch 0
Epoch 0	Train Loss: 46840.375  Val Loss: 45452.352



val loss: 45391.438: 100%|██████████| 7/7 [00:01<00:00,  5.98it/s]


Weight saved: epoch 1
Epoch 1	Train Loss: 45454.059  Val Loss: 45384.770



val loss: 45365.742: 100%|██████████| 7/7 [00:01<00:00,  6.02it/s]


...

Epoch 297	Train Loss: 44806.801  Val Loss: 44432.332



val loss: 43674.934: 100%|██████████| 7/7 [00:01<00:00,  6.23it/s]


Epoch 298	Train Loss: 44779.508  Val Loss: 44804.254



val loss: 43618.391: 100%|██████████| 7/7 [00:01<00:00,  6.08it/s]

Epoch 299	Train Loss: 44978.648  Val Loss: 44744.930

